In [2]:
import os

In [3]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project\\reasearch'

In [4]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updated_base_model_path:Path
    training_data: Path
    params_epochs:int
    params_learning_rate:float
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size: list
    
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_rot_log_dir:Path
    checkpoint_model_filepath:Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
            config = self.config.prepare_callbacks
            model_chkt_dir = os.path.dirname(config.checkpoint_model_filepath)
            create_directories([
                Path(model_chkt_dir),
                Path(config.tensorboard_root_log_dir)
            ])
            prepare_callback_config = PrepareCallbacksConfig(
                root_dir = Path(config.root_dir),
                tensorboard_rot_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
            
            return prepare_callback_config
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"../data_ingestion")
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_learning_rate = params.LEARNING_RATE,
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGEMENTATION,
            params_image_size=params.IMAGE_SIZE
            
        )
        return training_config

In [8]:
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
tf.config.run_functions_eagerly(True)  # Optional: helps debug data issues

In [9]:
class PrepareCallback:
    def __init__(self,config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y.%m.%d.%H.%M.%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_rot_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    
    @property
    def _create_cktp_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
        
    def get_tb_ckpt_callback(self):
        return[
            self._create_tb_callbacks,
            self._create_cktp_callback
        ]

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )
    def train_valid_generator(self):
        datagen_kwargs = dict(rescale=1./255, validation_split=0.20)
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            class_mode = "categorical",
        )
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
        self.valid_generator = valid_datagen.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = True,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                horizontal_flip=True,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = valid_datagen
            
        self.train_generator = train_datagen.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(model:tf.keras.Model, path:Path)->None:
        model.save(path)
    
    def train(self, callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
            
        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            validation_data = self.valid_generator,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            callbacks = callback_list
        )
            
        self.save_model(self.model, self.config.trained_model_path)

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()
    
    training_config = config.get_training_config()
    training = Training(config = training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
except Exception as e:
    raise e

[2025-12-03 05:56:11,855: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-03 05:56:11,876: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-03 05:56:11,886: INFO: common: created directory at : artifacts]
[2025-12-03 05:56:11,892: INFO: common: created directory at : artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-03 05:56:11,896: INFO: common: created directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-12-03 05:56:11,903: INFO: common: created directory at : artifacts\training]


[2025-12-03 05:56:13,542: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


c:\Users\MY LENOVO\anaconda3\envs\chicken\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0